# Inverse Kinematics and Singularities

Most end goals of these topics are to be able to position and orient the robot's end affector to a specified state.

__Forward Kinemtics__

$$\dot q = (\dot q_1, \dot q_2, \dot q_3, \dots, \dot q_n) \rightarrow \dot X (\dot x, \dot y, \dot z)$$

__Inverse Kinemtics__

$$\dot X (\dot x, \dot y, \dot z) \rightarrow \dot q (\dot q_1, \dot q_2, \dot q_3, \dots, \dot q_n)$$

The relationship between end effector velocity and joint velocities are

$$\xi = J(q) \dot(q)$$

Where $\xi$ are the end effector velocities

And we can invert this equation to obtain the joint velocities which would be needed to produce the desired end affector velocities. But this is only obvious when $J(q)$ is invertible.

We know that the Jacobian is invertible when it is full rank, IE when it is square. 

We can define a similar operation for any square matrix of the form $Ax = b$. But this doesn't always have a solution. We may have a unique solution, infinitely many solutions, or no solution at all. This can be important for finding positions, orientations, or velocities outside of the workspace of the robot.

## Important Aside

We often need to determine angles for end affector orientation, and we often replace the use of atan for atan2, which disambiguates the quadrant of the angle, since there are always 2 possible angles given when using atan.

## Inverse Kinematics

### Geometric Method

We can define the geometric constraints of the robot and try to use trigonometry to define what the joint space paramters must be depending on what the desired end affector space is.


Suppose we know the position and orientation of the end affector as either

$$[x, y, z, \theta, \rho, \phi]^T $$

or

$$
\begin{bmatrix}
R_d & o_d\\
0 & 1\\
\end{bmatrix}

\in SE(3)
$$

Then the desired position and orientation of the end effector is specified, in general by the six component vector, $[x, y, z, \theta, \rho, \phi]^T $ , yet the actual position ad orientation of the nd effector is determined by the $n$ component vector $[q_1, q_2 \dots, q_n]$ 


The number of equations we need to solve can quickly grow

We can use 6 contraint equations to reduce the nine variables (elements of the rotation matrix) to specify orientation down to three. WE must solve 12 nonlinear equations in n unknowns to determine the joint variables that are needed to position and orient the end affector. Even if the equations were linear, we would have 3 possibilities

- no solution - especially when n < 12
- many solutions - an infinite number in the linear case
- a uniqie solution - easy to determine in the linear case

The best type of solution is in closed form

- this has benefits for situations in which the end effector has to move quicky and these movements are faster to compute with closed form solutions
- when there are multiple solutions, it is much easier to choose the best one

 Reminder, when a robot is full rank, it has 6 degrees of freedom in 3d space

 12 equations with 6 unknowns, so the jacobian will be hard to solve.


### Algorithmic Solution

An advantage of the algorithmic solution is that it requires almost no insight on the part of the user.

However several things may go wrong
 
- no solution may be found
- the solution that is found may not be the best one

We can avoid this by using some amount of insight of the robot and looking for a specialcase which wouls simplify the inverse kinematics. For example, kinematic decoupling for a sperical wrist


#### Kinematic Decoupling

The inverse Kinemtatics Problem is greatly simplified for manipulators having 6 joints with the last 3 joints coincedent. This simplification comes from decoupling the overll inversekinematics problem into two independent subproblems

If we wanted to make sure that our robot had 6 degrees of freedom, then it would be a bad idea to just create a long chain of links because each of these joints would have an affect on the position of the end effector, and we would have to redo the inverse kinematics for each joint.

If we were to have a spherical wrist on the end, then the last 3 joints would have no translational effect on the end effector, only orientational. However, in practice the end effector would always have some offest from the center of the spherical wrist, thus giving the shperical joint some effect on the position of the end effector.

First we can solve for the position and orientation of the robot assuming that the d offset of the tool frame from the spheriacal wrist is 0, and then by recorrecting assuming that we have some actual physical offset.

Let us assume that

- o_c^0 represents the position in terms of the world frame, of the center of the spherical wrist
- d_6 is the offset of the end affector from the center of the spherical wrist
- the direction of the offset is in line with the z axis of the final joint of the spherical wrist

$$\textrm{Then, } o_d = o_c^0 + d_6 * R_6^0 \begin{bmatrix} 0\\ 0\\ 1\\ \end{bmatrix}$$

Remember that the solution is only simple when J(q) is invertible. And J(q) is only invertible when it is full rank. This means that if J(q) is a mxn matrix, that rank J(q) == rank [J(q)|b] where b is an m long vector. In other words, any extra column b would have to be a linear combination of the other columns of J(q). 

Applying this to the problem of inverse velocity kinemtics we can see that there is a solution when rank J(q) = rank [J(q)|$\xi$]

### The right pseudo inverse

Suppose the Jacobian is not Square. We either have more or less actuators than we do dimensions

Suppose A is an m x n matrix, m < n, and rank(A) = m.Then the result of $A \dot A^T$ is of size m x m and invertible.

Therefore flllowing the relation of the original, inverse, and identity matrices, we can see that 

$$
I_{mxm} = (AA^T)(AA^T)^{-1} \\
\\
I_{mxm} = A(A^T(AA^T)^{-1})\\
$$

Where $(A^T(AA^T)^{-1}$ is the right pseudo inverse


We use the right pseudo inverse when m < n, and then the left pseudo inverse when n > m

And remember that what this does is, given $\xi = J(q) \dot{q}$, then $J^{rp} \xi= \dot(q)$

## Singularities

Situations in hich the robot loses rank or degrees of freedom. When the determinant of the jacobian is 0, then there is a singularity because no inverse exists. In the physical realm this translates to a joint configuration in which the end affector, when a small amount of $\delta{q}$ is applied from any joint, loses a degree of freedom that no other joint can replace.

### How do we identify singularities

Calculate the jacobian of each orientation, and if 0, then we have a singularity. Singularities often separate regions of movement of the robot. IE there would be a singularity at the turning point in between an elbow up and an elbow down movement.

## Euler Angles

Given an Arbitrary Frame 0 with x,y,z axes 

1. Rotate the frame about the z axis by an angle $\theta $
2. Rotate the frame about the y axis by an angle $\phi $
3. Rotate the frame about the new z axis by an angle $\psi $

It is well known that every general rotation matrix can be constructed this way, but a proof requires finding the three angles, given only the rotation matrix R. SPelled out on pages 54-56 of the textbook, however it is not always possible to find an answer

## Fixed Angle Representation
Given a frame with x, y, z axes if, a roll is defined as

1. Rotate the frame about the x axis by an angle $\psi $
2. Rotate the frame about the y axis by an angle $\theta $
3. Rotate the frame about the z axis by an angle $\phi $

That way $R = R_{z, \phi} R_{y, \theta} R_{x, \psi}$

### Gimbal Lock

This is another form of singularity thet affects spherical wrists or other flying objects.